# Build, inspect and customize computational models using xarray-simlab

This notebook shows how the xarray-simlab's modelling framwork can be used to build a new model from scratch and further inspect or customize it. The example below consists of building a simple but extensible Landscape Evolution Model (LEM).

We start by importing some packages.

In [ ]:
import numpy as np
import xsimlab as xs

## Create model components

The xarray-simlab's modelling framwork works best when models are built from many small, pluggable components. This way those models can be further customized very easily.

Each component consists of a succint Python class decorated with ``xsimlab.process`` and in which we declare some variables as class attributes as well as a few methods implementing some computation at different stages of the simulation.

For more details on how to build model components, see xarray-simlab's [documentation](http://xarray-simlab.readthedocs.io/en/latest/create_model.html).

For simplicity, the actual computation code may be missing in some of the components below.

We start here by defining the model's grid, which is itself implemented as a component. It just computes the `x` and `y` coordinates at the beginning of the simulation from given grid `size` and node `spacing`. 

In [ ]:
@xs.process
class Grid2D(object):
    """Creates a regular (square) 2D grid."""
    size = xs.variable(description='nb. of grid nodes in x and y')
    spacing = xs.variable(description='grid node spacing in x and y')
    
    x = xs.variable(dims='x', intent='out', description='grid x coordinate')
    y = xs.variable(dims='y', intent='out', description='grid y coordinate')
    
    def initialize(self):
        self.x = np.linspace(0, (size - 1) * spacing, size)
        self.y = np.linspace(0, (size - 1) * spacing, size)

Boundary conditions have also their own component.

In [ ]:
@xs.process
class ClosedBoundaryFaces(object):
    """Set closed boundaries at each face of the grid."""
    size = xs.foreign(Grid2D, 'size')
    active_nodes = xs.variable(dims=('y', 'x'), intent='out')
    
    def initialize(self):
        self.active_nodes = np.ones((self.size, self.size), dtype=bool)
        self.active_nodes[[0, -1], :] = False
        self.active_nodes[:, [0, -1]] = False


Unlike a more "linear" way of writing models (i.e., writing code from top to bottom following the computation steps), with xarray-simlab's we usually start by implementing the most generic components first (which makes sense as we'd like to implement a modular model). These components are often executed at the end of the computation chain.

For example, here below we add three components that (1) combine all erosion processes, (2) combine all uplift processes (e.g., model forcing, isostatic rebound, flexure, etc.) and (3) updates elevation at each time step. 

In [ ]:
@xs.process
class TotalErosion(object):
    """Combine (sum) all erosion processes."""
    erosion = xs.variable(dims=('y', 'x'), intent='out')
    erosion_vars = xs.group('erosion')

    def run_step(self, *args):
        self.erosion = sum((err for err in self.erosion_vars))

In [ ]:
@xs.process
class TotalRockUplift(object):
    """Combine (sum) all uplift processes."""
    uplift = xs.variable(dims=('y', 'x'), intent='out')
    uplift_vars = xs.group('uplift')

    def run_step(self, *args):
        self.uplift = sum((u for u in self.uplift_vars))

In [ ]:
@xs.process
class Topography(object):
    """Update topographic erosion as a balance between uplift and
    erosion.
    
    Also compute derived variables (e.g., slope) on demand.

    """
    elevation = xs.variable(dims=('y', 'x'), intent='inout',
                            description='topographic elevation',
                            attrs={'units': 'm', 'math_symbol':'$z$'})
    total_erosion = xs.foreign(TotalErosion, 'erosion')
    total_uplift = xs.foreign(TotalRockUplift, 'uplift')
    slope = xs.on_demand(dims=('y', 'x'))

    def initialize(self):
        self.elevation_change = np.zeros_like(self.elevation)

    def run_step(self, *args):
        self.elevation_change = self.total_uplift - self.total_erosion

    def finalize_step(self):
        self.elevation += self.elevation_change
    
    @slope.compute
    def _compute_slope(self):
        # return slope computed from self.elevation
        return

The components below implement flow routing and erosion processes that we want to add to the model.

In [ ]:
@xs.process
class FlowRouterD8(object):
    """Route flow through topographic surface and
    compute flow tree.
    
    """
    size = xs.foreign(Grid2D, 'size')
    spacing = xs.foreign(Grid2D, 'spacing')
    active_nodes = xs.foreign(ClosedBoundaryFaces, 'active_nodes')
    elevation = xs.foreign(Topography, 'elevation')

    receivers = xs.variable(dims=('y', 'x'), intent='out')
    dist2receivers = xs.variable(dims=('y', 'x'), intent='out')
    ndonors = xs.variable(dims=('y', 'x'), intent='out')
    donors = xs.variable(dims=('y', 'x', 'd8'), intent='out')
    stack = xs.variable(dims=('y', 'x'), intent='out')
    
    def initialize(self):
        # initialize all working and output arrays
        pass
    
    def run_step(self):
        # route flow and compute flow tree (stack)
        pass

In [ ]:
@xs.process
class PropagateArea(object):
    """Computes drainage area."""
    spacing = xs.foreign(Grid2D, 'spacing')
    receivers = xs.foreign(FlowRouterD8, 'receivers')
    stack = xs.foreign(FlowRouterD8, 'stack')
    
    drainage_area = xs.variable(dims=('y', 'x'), intent='out',
                                description='drainage area')
    
    def initialize(self):
        # initialize self.drainage_area
        pass
    
    def run_step(self, dt):
        # updates self.drainage_area
        pass

In [ ]:
@xs.process
class StreamPower(object):
    """River bedrock erosion using Stream-Power law."""
    k_coef = xs.variable(description='stream-power constant')
    m_exp = xs.variable(description='stream-power drainage area exponent')
    n_exp = xs.variable(description='stream-power slope exponent')
    
    erosion = xs.variable(dims=('y', 'x'), intent='out', group='erosion')

    receivers = xs.foreign(FlowRouterD8, 'receivers')
    dist2receivers = xs.foreign(FlowRouterD8, 'dist2receivers')
    stack = xs.foreign(FlowRouterD8, 'stack')
    drainage_area = xs.foreign(PropagateArea, 'drainage_area')
    elevation = xs.foreign(Topography, 'elevation')
    
    def run_step(self, dt):
        # computes self.erosion
        pass

In [ ]:
@xs.process
class LinearDiffusion(object):
    """Hillslope erosion by diffusion."""
    k_coef = xs.variable(description='diffusivity')
    erosion = xs.variable(dims=('y', 'x'), intent='out', group='erosion')
    
    elevation = xs.foreign(Topography, 'elevation')
    
    spacing = xs.foreign(Grid2D, 'spacing')
    nx = xs.foreign(Grid2D, 'size')
    ny = xs.foreign(Grid2D, 'size')
    
    def run_step(self, dt):
        # computes self.erosion
        pass

In [ ]:
@xs.process
class BlockUplift(object):
    u_coef = xs.variable(dims=[(), ('y', 'x')], description='uplift rate')
    active_nodes = xs.foreign(ClosedBoundaryFaces, 'active_nodes', intent='in')
    
    uplift = xs.variable(dims=[(), ('y', 'x')], intent='out', group='uplift')
    
    def initialize(self):
        self.uplift = self.u_coef
        self.uplift[~self.active_nodes] = 0.

## Create new models

Once we have all components, creating a model is very easy: we just need to provide a dictionary with the decorated Python classes.

We don't need to worry about process dependencies, computation order, which variables are model inputs, etc. This is all inferred automatically from the components at model creation.

In [ ]:
model = xs.Model(
    {'grid': Grid2D,
     'boundaries': ClosedBoundaryFaces,
     'block_uplift': BlockUplift,
     'flow_routing': FlowRouterD8,
     'area': PropagateArea,
     'spower': StreamPower,
     'diffusion': LinearDiffusion,
     'erosion': TotalErosion,
     'uplift': TotalRockUplift,
     'topography': Topography}
)

## Inspect models

A `xsimlab.Model` object can be used to run simulations and also to interactively inspect the model's content (components, variables, dependencies, metadata). Actually, a `xsimlab.Model` object provides easy access to all the information required to automate many things like documentation, command-line interface, graphical interface, etc.

The representation (repr) of a `xsimlab.Model` object shows all the processes and model inputs:

In [ ]:
model

Each component (process) can be accessed like an attribute. It shows all declared variables and the implemented simulation stages, e.g.,

In [ ]:
model.grid

`xsimlab.Model` also supports dict-like access, e.g.,

In [ ]:
model['spower']

Documentation (docstrings) has been automatically added for each variable, e.g.,

In [ ]:
model.topography.elevation?

It is also possible to visualize a model (i.e., its components) as an Acyclic Directed Graph (DAG):

In [ ]:
model.visualize()

With model inputs:

In [ ]:
model.visualize(show_inputs=True)

With all variables:

In [ ]:
model.visualize(show_variables=True)

Or show only one variable (and all related foreign variables):

In [ ]:
model.visualize(show_only_variable=('topography', 'elevation'))

In [ ]:
model.visualize(show_only_variable=('erosion', 'erosion_vars'))

## Customize models

It is very easy to customize existing models by adding, replacing or removing components.

Here below, we create a new component to initialize simulations with a flat topography.

In [ ]:
@xs.process
class InitTopographyFlat(object):
    size = xs.foreign(Grid2D, 'size')
    elevation = xs.foreign(Topography, 'elevation', intent='out')
    
    def initialize(self):
        self.elevation = np.random.rand(self.size, self.size)

Then we add this component to the model created above. This creates a new `xsimlab.Model` object. Process dependencies, model inputs, etc. are all updated automatically.

In [ ]:
model2 = model.update_processes({'init': InitTopographyFlat})

Note that compared to the previous model elevation is not an input anymore:

In [ ]:
model2

As a second example, we create a new component that compute erosion coefficient depending on given rock resistivity.

In [ ]:
@xs.process
class Lithology(object):
    rock_resistivity = xs.variable(('z', 'y', 'x'),
                                   description='rock resistance to erosion')
    
    k_diff = xs.foreign(LinearDiffusion, 'k_coef', intent='out')
    k_sp = xs.foreign(StreamPower, 'k_coef', intent='out')

In [ ]:
model3 = model2.update_processes({'lithology': Lithology})

Note that the erosion coefficient are not model inputs anymore, instead `rock_resistivity` is added as another input:

In [ ]:
model3.visualize(show_inputs=True)

In the example below, we go back to a very simple version of the model (only uplift and erosion by diffusion) by removing all unneccessary components from the previous model.

In [ ]:
model4 = model3.drop_processes(['lithology', 'spower', 'flow_routing', 'area'])

In [ ]:
model4.visualize(show_inputs=True)